Each directory should contain three files:
- `attribute.txt`
- `graph_ic.inf`
- `graph.txt`

The file `attribute.txt` has a format like this:
```
n=500
m=1689
```
where `n` is the number of nodes and `m` is the number of edges

The file `graph_ic.inf` has the following format:
```
0 1 0.2
0 2 0.16666666666666666
1 0 0.25
...
...
498 493 0.125
499 498 0.25
499 495 0.125
```
where the first and second numbers are the node ids, and the third number is the probability of the directed edge. 

The file `graph.txt` is the same as `graph_ic.inf`, except that the third column is dropped:
```
0 1
0 2
1 0
...
...
498 493 
499 498
499 495
```

In [1]:
import json, os, pickle
from os.path import join as ojoin

In [2]:
with open('attribute_values.json') as f:
    attribute_values = json.load(f)

In [3]:
data_dir = os.path.abspath(ojoin('..','..','..','code','fairim','networks'))
ODIR = os.path.abspath(ojoin('..', 'datasets'))

In [4]:
def create_files(graph_number, attribute, value):
    graph_name = 'graph_spa_500_%d'%graph_number

    output_dir = ojoin(ODIR, graph_name, attribute, value)

    graph_fpath = ojoin(data_dir, '%s.pickle'%graph_name)
    with open(graph_fpath, 'rb') as f:
        graph = pickle.load(f)    
        
    subgraph_nodes = [node_id for node_id, attributes in graph.nodes(data=True) 
                      if attributes[attribute] == value]  

    index2networkx = dict()
    networkx2index = dict()
    for i, n in enumerate(subgraph_nodes):
        index2networkx[i] = n
        networkx2index[n] = i
        
    mapping = dict(id2nx=index2networkx, nx2id=networkx2index)
    mapping_file = ojoin(output_dir, 'mapping.json')
    with open(mapping_file, 'w') as f:
        json.dump(mapping, f)
    
    subgraph = graph.subgraph(subgraph_nodes)  

    attribute_file = ojoin(output_dir, 'attribute.txt')

    num_nodes = len(subgraph_nodes)
    num_edges = subgraph.number_of_edges()

    with open(attribute_file, 'w') as f:
        print('n=%d'%num_nodes, file=f)
        print('m=%d'%num_edges, file=f) 
        
    graph_file = ojoin(output_dir, 'graph.txt')
    ic_file = ojoin(output_dir, 'graph_ic.inf')

    with open(graph_file, 'w') as gf:
        with open(ic_file, 'w') as icf:
            for u, v, d in subgraph.edges(data=True):
                n1 = networkx2index[u]
                n2 = networkx2index[v]
                p = d['p']
                print('%d %d'%(n1, n2), file=gf)
                print('%d %d %g'%(n1, n2, p), file=icf)        

In [5]:
for graph_number in range(24):
    for attribute in attribute_values:
        for value in attribute_values[attribute]:
            create_files(graph_number, attribute, value)